In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import json
import numpy as np
import math
from nltk import word_tokenize
import torchtext
# from transformers import BertTokenizer, BertModel
from torchtext.models import RobertaClassificationHead, XLMR_BASE_ENCODER
# from torchtext.functional import to_tensor

batch_size = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
max_length = 512
train_data = []
train_inp_file = 'processed_data/GCDC/Clinton_train.jsonl'
with open(train_inp_file, 'r') as f:
    for line in f:
        json_obj = json.loads(line)
        # put <EOS> at the end of each sentence and add each sentence to the list
        for i in range(len(json_obj['sentences'])):
            # json_obj['sentences'][i].append('<EOS>')
            # truncate each sentence to max_length
            json_obj['sentences'][i] = json_obj['sentences'][i][:max_length]
            # # add padding to each sentence
            # json_obj['sentences'][i] = json_obj['sentences'][i] + ['[PAD]'] * (max_length - len(json_obj['sentences'][i]))

        # merge all sentences into one
        document = " ".join([word for sentence in json_obj['sentences'] for word in sentence])
        document = document.lower()
        label = json_obj['label']
        train_data.append((document, label-1))
   

test_data = []
test_inp_file = 'processed_data/GCDC/Clinton_test.jsonl'
with open(test_inp_file, 'r') as f:
    for line in f:
        json_obj = json.loads(line)
        # put <EOS> at the end of each sentence and add each sentence to the list
        for i in range(len(json_obj['sentences'])):
            # json_obj['sentences'][i].append('<EOS>')
            # truncate each sentence to max_length
            json_obj['sentences'][i] = json_obj['sentences'][i][:max_length]
            # # add padding to each sentence
            # json_obj['sentences'][i] = json_obj['sentences'][i] + ['[PAD]'] * (max_length - len(json_obj['sentences'][i]))
        # merge all sentences into one
        document = " ".join([word for sentence in json_obj['sentences'] for word in sentence])
        document = document.lower()
        label = json_obj['label']
        test_data.append((document, label-1))
        
print(len(train_data))
print(train_data[0])
print(train_data[1])
print(train_data[2])
print(len(test_data))
print(test_data[0])

800
('two options the us views the transitional national council as the sole / only legitimate interlocutor of the libyan people during this interim period , as libyans come together to plan their own future and a permanent , inclusive constitutional system that protects the rights of all libyans . this is in contrast to the qadhafi regime , which has lost all legitimacy to rule . the us views the transitional national council as the legitimate interlocutor of the libyan people during this interim period , as libyans come together to plan their own future and a permanent , inclusive constitutional system that protects the rights of all libyans . this is in contrast to the qadhafi regime , which has lost all legitimacy to rule . the inc is the institution through which we are engaging the libyan people at this time .', 2)
("ambassador , we just received an email from the adoption service provider about these cases . i am currently reviewing the files to determine if they qualify within 

In [3]:
# load the tokenizer
text_transform = XLMR_BASE_ENCODER.transform()

def apply_transform(doc):
    # also add padding to the end of the document
    return text_transform(doc[0]), doc[1]

train_dataset_unpadded = list(map(apply_transform, train_data))
test_dataset_unpadded = list(map(apply_transform, test_data))

# print(len(train_dataset_proto))
# print(train_dataset_proto[0])

# train_dataset = to_tensor(train_dataset_proto, padding_value=1)
# test_dataset = to_tensor(test_dataset_proto, padding_value=1)

print(len(train_dataset_unpadded[0][0]))
print(len(train_dataset_unpadded[1][0]))
print(len(train_dataset_unpadded[2][0]))
print(train_dataset_unpadded[0])
print(train_dataset_unpadded[1])
print(train_dataset_unpadded[2])
print(test_dataset_unpadded[0])


# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert = BertModel.from_pretrained('bert-base-uncased')

# print(tokenizer.vocab_size)
# print(bert.config.hidden_size)

# max_pad_len = 512
# pad_id = 0

# class GCDCDataset(Dataset):
#     def __init__(self, data, tokenizer):
#         self.data = data
#         self.tokenizer = tokenizer
        
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, idx):
#         document, label = self.data[idx]
#         tokenized_document = self.tokenizer.tokenize(document)
#         indexed_document = self.tokenizer.convert_tokens_to_ids(tokenized_document)
#         indexed_document = indexed_document[:max_pad_len]
#         indexed_document = indexed_document + [pad_id] * (max_pad_len - len(indexed_document))
#         return torch.tensor(indexed_document), torch.tensor(label)


186
275
255
([0, 6626, 50717, 70, 1821, 16912, 70, 149307, 289, 15889, 215394, 237, 70, 31247, 248, 4734, 148436, 67, 230415, 111, 70, 25474, 1643, 3395, 20271, 903, 1940, 464, 14922, 6, 4, 237, 25474, 1643, 7, 1380, 25842, 47, 1774, 2363, 10002, 22690, 136, 10, 28123, 6, 4, 55128, 171484, 289, 5426, 450, 59959, 7, 70, 38109, 111, 756, 25474, 1643, 7, 6, 5, 903, 83, 23, 69822, 47, 70, 44448, 35824, 14, 63647, 6, 4, 3129, 1556, 72856, 756, 148436, 2408, 47, 79986, 6, 5, 70, 1821, 16912, 70, 149307, 289, 15889, 215394, 237, 70, 148436, 67, 230415, 111, 70, 25474, 1643, 3395, 20271, 903, 1940, 464, 14922, 6, 4, 237, 25474, 1643, 7, 1380, 25842, 47, 1774, 2363, 10002, 22690, 136, 10, 28123, 6, 4, 55128, 171484, 289, 5426, 450, 59959, 7, 70, 38109, 111, 756, 25474, 1643, 7, 6, 5, 903, 83, 23, 69822, 47, 70, 44448, 35824, 14, 63647, 6, 4, 3129, 1556, 72856, 756, 148436, 2408, 47, 79986, 6, 5, 70, 23, 238, 83, 70, 38016, 8305, 3129, 642, 621, 44173, 9966, 70, 25474, 1643, 3395, 99, 903, 1733,

In [4]:
max_pad_len = 512
pad_id = 1

# pad the document to max_pad_len
def pad_document(document):
    return document + [pad_id] * (max_pad_len - len(document))

train_dataset = list(map(lambda x: (pad_document(x[0]), x[1]), train_dataset_unpadded))
test_dataset = list(map(lambda x: (pad_document(x[0]), x[1]), test_dataset_unpadded))

print(len(train_dataset[0][0]))
print(len(train_dataset[1][0]))
print(len(train_dataset[2][0]))
print(train_dataset[0])
print(train_dataset[1])
print(train_dataset[2])
print(test_dataset[0])

512
512
512
([0, 6626, 50717, 70, 1821, 16912, 70, 149307, 289, 15889, 215394, 237, 70, 31247, 248, 4734, 148436, 67, 230415, 111, 70, 25474, 1643, 3395, 20271, 903, 1940, 464, 14922, 6, 4, 237, 25474, 1643, 7, 1380, 25842, 47, 1774, 2363, 10002, 22690, 136, 10, 28123, 6, 4, 55128, 171484, 289, 5426, 450, 59959, 7, 70, 38109, 111, 756, 25474, 1643, 7, 6, 5, 903, 83, 23, 69822, 47, 70, 44448, 35824, 14, 63647, 6, 4, 3129, 1556, 72856, 756, 148436, 2408, 47, 79986, 6, 5, 70, 1821, 16912, 70, 149307, 289, 15889, 215394, 237, 70, 148436, 67, 230415, 111, 70, 25474, 1643, 3395, 20271, 903, 1940, 464, 14922, 6, 4, 237, 25474, 1643, 7, 1380, 25842, 47, 1774, 2363, 10002, 22690, 136, 10, 28123, 6, 4, 55128, 171484, 289, 5426, 450, 59959, 7, 70, 38109, 111, 756, 25474, 1643, 7, 6, 5, 903, 83, 23, 69822, 47, 70, 44448, 35824, 14, 63647, 6, 4, 3129, 1556, 72856, 756, 148436, 2408, 47, 79986, 6, 5, 70, 23, 238, 83, 70, 38016, 8305, 3129, 642, 621, 44173, 9966, 70, 25474, 1643, 3395, 99, 903, 1733,

In [5]:
# convert the dataset to tensors
train_dataset = list(map(lambda x: (torch.tensor(x[0]), torch.tensor(x[1])), train_dataset))
test_dataset = list(map(lambda x: (torch.tensor(x[0]), torch.tensor(x[1])), test_dataset))
# train_dataset = list(map(lambda x: (torch.tensor(x[0]), torch.tensor(x[1])), train_dataset_unpadded))
# test_dataset = list(map(lambda x: (torch.tensor(x[0]), torch.tensor(x[1])), test_dataset_unpadded))

print(len(train_dataset))
print(len(test_dataset))
print(train_dataset[0])
print(test_dataset[0])

800
200
(tensor([     0,   6626,  50717,     70,   1821,  16912,     70, 149307,    289,
         15889, 215394,    237,     70,  31247,    248,   4734, 148436,     67,
        230415,    111,     70,  25474,   1643,   3395,  20271,    903,   1940,
           464,  14922,      6,      4,    237,  25474,   1643,      7,   1380,
         25842,     47,   1774,   2363,  10002,  22690,    136,     10,  28123,
             6,      4,  55128, 171484,    289,   5426,    450,  59959,      7,
            70,  38109,    111,    756,  25474,   1643,      7,      6,      5,
           903,     83,     23,  69822,     47,     70,  44448,  35824,     14,
         63647,      6,      4,   3129,   1556,  72856,    756, 148436,   2408,
            47,  79986,      6,      5,     70,   1821,  16912,     70, 149307,
           289,  15889, 215394,    237,     70, 148436,     67, 230415,    111,
            70,  25474,   1643,   3395,  20271,    903,   1940,    464,  14922,
             6,      4,    237,

In [6]:
# # create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [7]:
num_classes = 3
input_dim = 768

classifier_head = RobertaClassificationHead(num_classes=num_classes, input_dim=input_dim)
model = XLMR_BASE_ENCODER.get_model(head=classifier_head)
model = model.to(device)

In [8]:
import torchtext.functional as F
from torch.optim import AdamW

learning_rate = 0.001
optim = AdamW(model.parameters(), lr=learning_rate)
criteria = nn.CrossEntropyLoss()

def train(model, train_loader, optim, criteria):
    model.train()
    total_loss = 0
    total_correct = 0
    total = 0
    for batch in train_loader:
        optim.zero_grad()
        x, y = batch
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x)
        loss = criteria(y_pred, y)
        loss.backward()
        optim.step()
        total_loss += loss.item()
        total_correct += (torch.argmax(y_pred, dim=1) == y).sum().item()
        total += len(y)
    return total_loss / total, total_correct / total

def test(model, test_loader, criteria):
    model.eval()
    total_loss = 0
    total_correct = 0
    total = 0
    with torch.no_grad():
        for batch in test_loader:
            x, y = batch
            x = x.to(device)
            y = y.to(device)
            y_pred = model(x)
            loss = criteria(y_pred, y)
            total_loss += loss.item()
            total_correct += (torch.argmax(y_pred, dim=1) == y).sum().item()
            total += len(y)
    return total_loss / total, total_correct / total



In [9]:
num_epochs = 10
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, optim, criteria)
    test_loss, test_acc = test(model, test_loader, criteria)
    print('Epoch: {}, Train Loss: {:.4f}, Train Acc: {:.4f}, Test Loss: {:.4f}, Test Acc: {:.4f}'.format(epoch, train_loss, train_acc, test_loss, test_acc))


OutOfMemoryError: CUDA out of memory. Tried to allocate 734.00 MiB (GPU 0; 5.80 GiB total capacity; 4.16 GiB already allocated; 527.44 MiB free; 4.34 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF